This notebook shows how to use [CDD](https://www.inf.ethz.ch/personal/fukudak/cdd_home/) to compute controlled invariant sets for an hybrid system.
We consider the `cruise_control.jl` example of HybridSystems.jl which comes from [this paper](https://dl.acm.org/citation.cfm?id=2461378).

In [ ]:
include(Pkg.dir("HybridSystems", "examples", "cruise_control.jl"));

In [ ]:
const va = 15.6
const vb = 24.5
const vc = 29.5
const v = (va, vb, vc)
const U = 4.
const m0 = 500
const T = 2
const N = 1
const M = 1
const H = 0.8;

In [ ]:
function liftu(S, sys::HybridSystems.DiscreteLinearControlSystem)
    [sys.A sys.B] \ S
end
function new_constraint(hs, S, q, t)
    @assert source(hs, t) == q
    σ = symbol(hs, t)
    r = target(hs, t)
    ABset = liftu(S[1], hs.resetmaps[σ])
    project(ABset, 1:statedim(hs, q))
end
function new_constraints(hs, S, q)
    map(t -> new_constraint(hs, S, q, t), out_transitions(hs, q))
end
function add_hrep!(S, h::HalfSpace)
    if h in S
        false
    else
        push!(S, SimpleHRepresentation(reshape(h.a, 1, length(h.a)), [h.β]))
        true
    end
end
function add_constraint!(S, P)
    added = count(map(h -> add_hrep!(S, h), ineqs(P))) + count(map(h -> add_hrep!(S, h), eqs(P)))
    @show nineqs(S)
    removehredundancy!(S)
    @show nineqs(S)
    added
end
function add_constraints!(S::Polyhedron, Ps::Vector{<:Polyhedron})
    sum(add_constraint!.(S, Ps))
end
function set_iteration!(hs, S)
    Ps = map(q -> new_constraints(hs, S, q), states(hs))
    added = add_constraints!.(S, Ps)
    @show added
end

In [ ]:
hs = cruise_control_example(N, M, vmin = 5., v=(va, vb, vc), U=U, H=H, sym=false, m0=500);
I0 = hs.invariants;
@show nineqs(I0[1])
I = copy(I0);

In [ ]:
set_iteration!(hs, I);
@show nineqs(I[1])
#set_iteration!(hs, I);
#set_iteration!(hs, I);

In [ ]:
using Plots
pyplot()

In [ ]:
plot(project(I0[1], 1:2, :FourierMotzkin))

In [ ]:
project(I0[1], 3:4)

In [ ]:
x

In [ ]:
removehredundancy!(x)

In [ ]:
x